In [1]:
import pandas as pd
import numpy as np

import holidays
from collections import Counter
#from datetime import timedelta, date
import statsmodels
from sklearn.preprocessing import LabelEncoder
from statsmodels.formula.api import ols
from scipy import stats
import seaborn as sns

from tqdm import tqdm
import itertools

from scipy.spatial.distance import euclidean
import time
#%matplotlib inline
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing


/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
x_train = pd.read_csv('/Users/s0c02nj/Desktop/merch_ts/mode_imp_all.csv')
x_test =  pd.read_csv('/Users/s0c02nj/Desktop/merch_ts/test_data_date.csv')

In [3]:
x_train['ind'] = 'tr'
x_test['ind'] = 'te'

In [4]:
df_train = x_train[['mds_fam_id','dc_nbr','date','dept_nbr','vendor_nm',
                   'unit_cost','corp_retail_price','ind']]

y_train = x_train[['actual_packs_order']]


df_test = x_test[['mds_fam_id','dc_nbr','date','dept_nbr','vendor_nm',
                   'unit_cost','corp_retail_price','ind']]

In [5]:
df_total = pd.concat([df_train,df_test])

In [6]:
df_total['date'] = pd.to_datetime(df_total['date'])

df_total['week'] =  df_total['date'].dt.week
df_total['year'] =  df_total['date'].dt.year
df_total['month'] = df_total['date'].dt.month


In [7]:
df_total['mds_dc'] = df_total['mds_fam_id'].astype(str) + '_' + df_total['dc_nbr'].astype(str)
df_total['dept_dc'] = df_total['dc_nbr'].astype(str) + '_' + df_total['dept_nbr'].astype(str)
df_total['dept_mds'] = df_total['mds_fam_id'].astype(str) + '_' + df_total['dept_nbr'].astype(str)

In [8]:
df_total['mds_week'] = df_total['mds_fam_id'].astype(str) + '_' + df_total['week'].astype(str)

In [9]:
y_p1 = list(y_train['actual_packs_order'].values)
y_p2 = [0]*x_test.shape[0]
y_p = y_p1 + y_p2

df_total['order'] = y_p

In [10]:
df_train1 = df_total[df_total['ind'] == 'tr']

In [11]:
cat_cols = ['mds_fam_id',
             'dc_nbr',
             'dept_nbr',
             'week',
             'year',
             'month',
             'mds_dc',
             'dept_dc',
             'dept_mds',
             'mds_week']

In [12]:
for i,col in tqdm(enumerate(cat_cols)):
    med = df_train1.groupby(col)['order'].median()
    df_total[str(col)+'count'+'1'] = df_total[col].map(med)

10it [00:00, 127.39it/s]


In [13]:
df_total1 = df_total.sort_values(by=['mds_dc','date'])

In [14]:
df_total1.index = range(0,len(df_total1))

In [15]:
#df_total1
unique_md_dc = list(df_total1['mds_dc'].unique())

In [16]:
df_total1.columns

Index(['mds_fam_id', 'dc_nbr', 'date', 'dept_nbr', 'vendor_nm', 'unit_cost',
       'corp_retail_price', 'ind', 'week', 'year', 'month', 'mds_dc',
       'dept_dc', 'dept_mds', 'mds_week', 'order', 'mds_fam_idcount1',
       'dc_nbrcount1', 'dept_nbrcount1', 'weekcount1', 'yearcount1',
       'monthcount1', 'mds_dccount1', 'dept_dccount1', 'dept_mdscount1',
       'mds_weekcount1'],
      dtype='object')

In [17]:
reqd_cols = ['mds_fam_id',
             'dc_nbr',
             'date',
             'dept_nbr',
             'ind',
             'week',
             'year',
             'month',
             'mds_dc',
             'dept_dc',
             'dept_mds',
             'mds_week',
             'mds_fam_idcount1',
             'dc_nbrcount1',
             'dept_nbrcount1',
             'weekcount1',
             'yearcount1',
             'monthcount1',
             'mds_dccount1',
             'dept_dccount1',
             'dept_mdscount1',
             'mds_weekcount1',
             'order']

In [18]:
df_total1 = df_total1[reqd_cols]

In [19]:
le = LabelEncoder()
le_cols = ['mds_dc','dept_dc','dept_mds','mds_week']

for col in tqdm(le_cols):
    le = LabelEncoder()
    #le = le.fit(df_total1[col])
    df_total1[col] = le.fit_transform(df_total1[col])

100%|██████████| 4/4 [00:00<00:00, 64.90it/s]


In [20]:
df_train_v1 = df_total1[df_total1['ind'] == 'tr']
df_test_v1 =  df_total1[df_total1['ind'] == 'te']

In [21]:
unique_md_dc = list(df_total1['mds_dc'].unique())

In [22]:
df_comb_tr = pd.DataFrame()
df_comb_val = pd.DataFrame()


x_t1 = df_train_v1

for u_id in tqdm(unique_md_dc) :
    
    #Groupby
    df_temp1 = x_t1[x_t1['mds_dc'] == u_id].sort_values(by='date')
    df_temp1 = df_temp1.drop(['ind','date'],axis=1)
    
    df_temp1.index = range(0,len(df_temp1))
    
    #Train-test split
    df_temp_tr = df_temp1[0:int(df_temp1.shape[0]*0.85)]
    df_temp_val = df_temp1[int(df_temp1.shape[0]*0.15):]
    #print (df_temp.head())
    
    #Appending
    df_comb_tr =  pd.concat([df_comb_tr,df_temp_tr],axis=0)
    df_comb_val = pd.concat([df_comb_val,df_temp_val],axis=0)

100%|██████████| 442/442 [00:09<00:00, 46.07it/s]


In [23]:
x_train = df_comb_tr.drop(['order'],axis=1)
y_train = df_comb_tr['order']

x_val = df_comb_val.drop(['order'],axis=1)
y_val = df_comb_val['order']

In [24]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'max_depth': 10, 
    'learning_rate': 0.1,
    'verbose': 1, 
    }

In [25]:
n_estimators = 1000

In [26]:
d_train = lgb.Dataset(x_train, label = y_train)
d_valid = lgb.Dataset(x_val, label = y_val)

In [27]:
model = lgb.train(params,
                  d_train, 
                  n_estimators,
                  valid_sets = [d_train, d_valid],
                  early_stopping_rounds=500,
                  verbose_eval=1)


[1]	training's rmse: 221.318	valid_1's rmse: 205.215
Training until validation scores don't improve for 500 rounds.
[2]	training's rmse: 207.577	valid_1's rmse: 193.004
[3]	training's rmse: 195.613	valid_1's rmse: 182.048
[4]	training's rmse: 185.042	valid_1's rmse: 172.653
[5]	training's rmse: 175.755	valid_1's rmse: 164.678
[6]	training's rmse: 168.023	valid_1's rmse: 157.938
[7]	training's rmse: 161.679	valid_1's rmse: 152.134
[8]	training's rmse: 155.824	valid_1's rmse: 147.18
[9]	training's rmse: 150.596	valid_1's rmse: 142.819
[10]	training's rmse: 146.4	valid_1's rmse: 139.191
[11]	training's rmse: 142.415	valid_1's rmse: 136.022
[12]	training's rmse: 139.316	valid_1's rmse: 133.325
[13]	training's rmse: 136.273	valid_1's rmse: 130.977
[14]	training's rmse: 133.465	valid_1's rmse: 128.844
[15]	training's rmse: 130.865	valid_1's rmse: 126.819
[16]	training's rmse: 129.108	valid_1's rmse: 125.412
[17]	training's rmse: 127.277	valid_1's rmse: 123.97
[18]	training's rmse: 125.352	va

[177]	training's rmse: 91.6201	valid_1's rmse: 97.6451
[178]	training's rmse: 91.5692	valid_1's rmse: 97.6166
[179]	training's rmse: 91.5552	valid_1's rmse: 97.6014
[180]	training's rmse: 91.5163	valid_1's rmse: 97.5681
[181]	training's rmse: 91.3283	valid_1's rmse: 97.5502
[182]	training's rmse: 91.3035	valid_1's rmse: 97.5288
[183]	training's rmse: 91.2674	valid_1's rmse: 97.5012
[184]	training's rmse: 91.2232	valid_1's rmse: 97.4613
[185]	training's rmse: 91.0598	valid_1's rmse: 97.5143
[186]	training's rmse: 91.0241	valid_1's rmse: 97.4819
[187]	training's rmse: 90.9097	valid_1's rmse: 97.3986
[188]	training's rmse: 90.8729	valid_1's rmse: 97.3659
[189]	training's rmse: 90.8632	valid_1's rmse: 97.3593
[190]	training's rmse: 90.702	valid_1's rmse: 97.3499
[191]	training's rmse: 90.6641	valid_1's rmse: 97.3145
[192]	training's rmse: 90.6359	valid_1's rmse: 97.2892
[193]	training's rmse: 90.5971	valid_1's rmse: 97.2572
[194]	training's rmse: 90.5535	valid_1's rmse: 97.231
[195]	traini

[351]	training's rmse: 83.5017	valid_1's rmse: 93.5943
[352]	training's rmse: 83.4808	valid_1's rmse: 93.5864
[353]	training's rmse: 83.417	valid_1's rmse: 93.5861
[354]	training's rmse: 83.395	valid_1's rmse: 93.5791
[355]	training's rmse: 83.3399	valid_1's rmse: 93.5343
[356]	training's rmse: 83.3155	valid_1's rmse: 93.5161
[357]	training's rmse: 83.2712	valid_1's rmse: 93.4833
[358]	training's rmse: 83.2514	valid_1's rmse: 93.4714
[359]	training's rmse: 83.2216	valid_1's rmse: 93.453
[360]	training's rmse: 83.1842	valid_1's rmse: 93.4251
[361]	training's rmse: 83.1672	valid_1's rmse: 93.414
[362]	training's rmse: 83.1432	valid_1's rmse: 93.4087
[363]	training's rmse: 83.1099	valid_1's rmse: 93.3894
[364]	training's rmse: 83.0614	valid_1's rmse: 93.372
[365]	training's rmse: 83.0087	valid_1's rmse: 93.372
[366]	training's rmse: 82.9819	valid_1's rmse: 93.3689
[367]	training's rmse: 82.9141	valid_1's rmse: 93.3689
[368]	training's rmse: 82.8864	valid_1's rmse: 93.3466
[369]	training's

[539]	training's rmse: 78.3335	valid_1's rmse: 90.7144
[540]	training's rmse: 78.3034	valid_1's rmse: 90.6941
[541]	training's rmse: 78.3021	valid_1's rmse: 90.6932
[542]	training's rmse: 78.2656	valid_1's rmse: 90.7046
[543]	training's rmse: 78.2403	valid_1's rmse: 90.683
[544]	training's rmse: 78.1956	valid_1's rmse: 90.6731
[545]	training's rmse: 78.1697	valid_1's rmse: 90.6576
[546]	training's rmse: 78.1462	valid_1's rmse: 90.6451
[547]	training's rmse: 78.0944	valid_1's rmse: 90.6268
[548]	training's rmse: 78.0815	valid_1's rmse: 90.6226
[549]	training's rmse: 78.0653	valid_1's rmse: 90.6144
[550]	training's rmse: 78.0412	valid_1's rmse: 90.605
[551]	training's rmse: 77.9631	valid_1's rmse: 90.5931
[552]	training's rmse: 77.9395	valid_1's rmse: 90.5815
[553]	training's rmse: 77.9072	valid_1's rmse: 90.5803
[554]	training's rmse: 77.8744	valid_1's rmse: 90.5722
[555]	training's rmse: 77.8573	valid_1's rmse: 90.5643
[556]	training's rmse: 77.8311	valid_1's rmse: 90.5469
[557]	traini

[715]	training's rmse: 74.1122	valid_1's rmse: 88.5351
[716]	training's rmse: 74.0804	valid_1's rmse: 88.511
[717]	training's rmse: 74.0611	valid_1's rmse: 88.5112
[718]	training's rmse: 74.0474	valid_1's rmse: 88.5079
[719]	training's rmse: 74.032	valid_1's rmse: 88.4942
[720]	training's rmse: 74.0185	valid_1's rmse: 88.4908
[721]	training's rmse: 73.9968	valid_1's rmse: 88.4767
[722]	training's rmse: 73.9614	valid_1's rmse: 88.4652
[723]	training's rmse: 73.9425	valid_1's rmse: 88.4559
[724]	training's rmse: 73.9233	valid_1's rmse: 88.4413
[725]	training's rmse: 73.8954	valid_1's rmse: 88.4268
[726]	training's rmse: 73.8817	valid_1's rmse: 88.4216
[727]	training's rmse: 73.8669	valid_1's rmse: 88.4094
[728]	training's rmse: 73.851	valid_1's rmse: 88.3989
[729]	training's rmse: 73.843	valid_1's rmse: 88.3941
[730]	training's rmse: 73.8273	valid_1's rmse: 88.3779
[731]	training's rmse: 73.7968	valid_1's rmse: 88.3578
[732]	training's rmse: 73.79	valid_1's rmse: 88.353
[733]	training's 

[893]	training's rmse: 70.7753	valid_1's rmse: 86.7798
[894]	training's rmse: 70.7538	valid_1's rmse: 86.762
[895]	training's rmse: 70.7496	valid_1's rmse: 86.7594
[896]	training's rmse: 70.7288	valid_1's rmse: 86.7441
[897]	training's rmse: 70.721	valid_1's rmse: 86.7422
[898]	training's rmse: 70.716	valid_1's rmse: 86.7402
[899]	training's rmse: 70.7017	valid_1's rmse: 86.7311
[900]	training's rmse: 70.6965	valid_1's rmse: 86.7284
[901]	training's rmse: 70.681	valid_1's rmse: 86.7166
[902]	training's rmse: 70.6696	valid_1's rmse: 86.7114
[903]	training's rmse: 70.6552	valid_1's rmse: 86.7
[904]	training's rmse: 70.6525	valid_1's rmse: 86.6991
[905]	training's rmse: 70.65	valid_1's rmse: 86.6977
[906]	training's rmse: 70.6398	valid_1's rmse: 86.6973
[907]	training's rmse: 70.635	valid_1's rmse: 86.6932
[908]	training's rmse: 70.6232	valid_1's rmse: 86.6906
[909]	training's rmse: 70.601	valid_1's rmse: 86.6797
[910]	training's rmse: 70.5776	valid_1's rmse: 86.6663
[911]	training's rmse

In [28]:
y_pred = model.predict(x_val, num_iteration=model.best_iteration)

In [29]:
from sklearn.metrics import r2_score
r2 = r2_score(y_val, y_pred)

In [30]:
r2

0.846235630359349

In [31]:
df_train_v1

,mds_fam_id,dc_nbr,date,dept_nbr,ind,week,year,month,mds_dc,dept_dc,...,dc_nbrcount1,dept_nbrcount1,weekcount1,yearcount1,monthcount1,mds_dccount1,dept_dccount1,dept_mdscount1,mds_weekcount1,order
0,105899148,6006,2016-11-26,46,tr,47,2016,11,0,3,...,15.0,14,32,46.5,28,32.0,18.0,32,32.0,4
1,105899148,6006,2016-12-03,46,tr,48,2016,12,0,3,...,15.0,14,20,46.5,22,32.0,18.0,32,23.5,14
2,105899148,6006,2016-12-10,46,tr,49,2016,12,0,3,...,15.0,14,20,46.5,22,32.0,18.0,32,24.5,10
3,105899148,6006,2016-12-17,46,tr,50,2016,12,0,3,...,15.0,14,28,46.5,22,32.0,18.0,32,30.0,20
4,105899148,6006,2016-12-24,46,tr,51,2016,12,0,3,...,15.0,14,30,46.5,22,32.0,18.0,32,32.0,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60404,94169512,7045,2019-06-29,46,tr,26,2019,6,441,302,...,15.0,14,32,32.0,32,4.0,18.0,5,5.0,4
60405,94169512,7045,2019-07-06,46,tr,27,2019,7,441,302,...,15.0,14,32,32.0,32,4.0,18.0,5,4.0,4
60406,94169512,7045,2019-07-13,46,tr,28,2019,7,441,302,...,15.0,14,32,32.0,32,4.0,18.0,5,5.0,5
60407,94169512,7045,2019-07-20,46,tr,29,2019,7,441,302,...,15.0,14,32,32.0,32,4.0,18.0,5,5.0,5


In [32]:
set(df_test_v1.columns)- set(x_val.columns) 

{'date', 'ind', 'order'}

In [33]:
df_test_v1.sort_values(by=['mds_dc','date'])

,mds_fam_id,dc_nbr,date,dept_nbr,ind,week,year,month,mds_dc,dept_dc,...,dc_nbrcount1,dept_nbrcount1,weekcount1,yearcount1,monthcount1,mds_dccount1,dept_dccount1,dept_mdscount1,mds_weekcount1,order
140,105899148,6006,2019-08-10,46,te,32,2019,8,0,3,...,15.0,14,32,32.0,32,32.0,18.0,32,32.0,0
141,105899148,6006,2019-08-24,46,te,34,2019,8,0,3,...,15.0,14,32,32.0,32,32.0,18.0,32,32.0,0
142,105899148,6006,2019-09-07,46,te,36,2019,9,0,3,...,15.0,14,32,32.0,32,32.0,18.0,32,32.0,0
143,105899148,6006,2019-09-21,46,te,38,2019,9,0,3,...,15.0,14,32,32.0,32,32.0,18.0,32,32.0,0
144,105899148,6006,2019-10-05,46,te,40,2019,10,0,3,...,15.0,14,24,32.0,24,32.0,18.0,32,32.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60415,94169512,7045,2019-09-14,46,te,37,2019,9,441,302,...,15.0,14,32,32.0,32,4.0,18.0,5,7.0,0
60416,94169512,7045,2019-09-21,46,te,38,2019,9,441,302,...,15.0,14,32,32.0,32,4.0,18.0,5,5.0,0
60417,94169512,7045,2019-09-28,46,te,39,2019,9,441,302,...,15.0,14,32,32.0,32,4.0,18.0,5,7.5,0
60418,94169512,7045,2019-10-05,46,te,40,2019,10,441,302,...,15.0,14,24,32.0,24,4.0,18.0,5,16.0,0


In [34]:
df_test_v2 = df_test_v1.drop(['date', 'ind', 'order'],axis=1)

In [35]:
y_pred_test = model.predict(df_test_v2, num_iteration=model.best_iteration)

In [36]:
y_pred_test

array([31.96930875, 31.17756331, 30.7180679 , ...,  7.79978682,
       17.68105335, 11.16430988])

In [37]:
y_pred_test = [0.1 if x < 0 else x for x in y_pred_test]

In [38]:
df_test_v1['order'] = y_pred_test

/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [39]:
df_test_v1.head()

,mds_fam_id,dc_nbr,date,dept_nbr,ind,week,year,month,mds_dc,dept_dc,...,dc_nbrcount1,dept_nbrcount1,weekcount1,yearcount1,monthcount1,mds_dccount1,dept_dccount1,dept_mdscount1,mds_weekcount1,order
140,105899148,6006,2019-08-10,46,te,32,2019,8,0,3,...,15.0,14,32,32.0,32,32.0,18.0,32,32.0,31.969309
141,105899148,6006,2019-08-24,46,te,34,2019,8,0,3,...,15.0,14,32,32.0,32,32.0,18.0,32,32.0,31.177563
142,105899148,6006,2019-09-07,46,te,36,2019,9,0,3,...,15.0,14,32,32.0,32,32.0,18.0,32,32.0,30.718068
143,105899148,6006,2019-09-21,46,te,38,2019,9,0,3,...,15.0,14,32,32.0,32,32.0,18.0,32,32.0,31.416503
144,105899148,6006,2019-10-05,46,te,40,2019,10,0,3,...,15.0,14,24,32.0,24,32.0,18.0,32,32.0,30.349884


In [40]:
df_test_v1['date'] = df_test_v1['date'].apply(lambda x : x.strftime("%Y-%m-%d (%H:%M:%S.%f)")[0:10])

/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [41]:
dict_map = {}

for i in tqdm(range(0,len(df_test_v1))):
    
    mds = df_test_v1['mds_fam_id'].iloc[i]
    dc =  df_test_v1['dc_nbr'].iloc[i]
    dt =  df_test_v1['date'].iloc[i]
    
    dict_map[(mds,dc,dt)] = df_test_v1['order'].iloc[i]

100%|██████████| 3903/3903 [00:00<00:00, 16919.84it/s]


In [42]:
dict_dt = {}

for i in tqdm(range(0,len(x_test))):
    
    dict_dt[x_test['wm_year_wk_nbr'].iloc[i]] = x_test['date'].iloc[i]

100%|██████████| 3903/3903 [00:00<00:00, 32375.22it/s]


In [80]:
final_test = pd.read_excel('/Users/s0c02nj/Desktop/test_merch.xlsx')

In [44]:
order = []

for i in tqdm(range(0,len(final_test))):
    
    mds = final_test['mds_fam_id'].iloc[i]
    dc =  final_test['dc_nbr'].iloc[i]
    dt =  dict_dt[final_test['wm_year_wk_nbr'].iloc[i]]
    
    order.append(dict_map[(mds,dc,dt)])
    

100%|██████████| 3903/3903 [00:00<00:00, 21570.81it/s]


In [45]:
order = [0.1 if x < 0 else x for x in order]

In [46]:
final_test['predicted_order_qty'] = order
#final_test['predicted_order_qty'] = final_test['predicted_order_qty'] * final_test['vnpk_qty']

In [47]:
#final_test['predicted_order_qty'] = final_test['predicted_order_qty'] * final_test['vnpk_qty']

In [ ]:
final_test.to_csv('/Users/s0c02nj/Desktop/merch_pred2.csv',index = False)

#### Time series component

In [48]:
x_train.columns

Index(['mds_fam_id', 'dc_nbr', 'dept_nbr', 'week', 'year', 'month', 'mds_dc',
       'dept_dc', 'dept_mds', 'mds_week', 'mds_fam_idcount1', 'dc_nbrcount1',
       'dept_nbrcount1', 'weekcount1', 'yearcount1', 'monthcount1',
       'mds_dccount1', 'dept_dccount1', 'dept_mdscount1', 'mds_weekcount1'],
      dtype='object')

In [49]:
df_train_v2 = df_train_v1.drop(['date', 'ind', 'order'],axis=1)

In [50]:
train_pred = model.predict(df_train_v2, num_iteration=model.best_iteration)

In [51]:
df_train_v1['pred_train'] = train_pred
df_train_v1['date'] = df_train_v1['date'].apply(lambda x : x.strftime("%Y-%m-%d (%H:%M:%S.%f)")[0:10])

/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [52]:
dict_map_tr = {}

for i in tqdm(range(0,len(df_train_v1))):
    
    mds = df_train_v1['mds_fam_id'].iloc[i]
    dc =  df_train_v1['dc_nbr'].iloc[i]
    dt =  df_train_v1['date'].iloc[i]
    
    dict_map_tr[(mds,dc,dt)] = df_train_v1['pred_train'].iloc[i]

100%|██████████| 56517/56517 [00:02<00:00, 21439.93it/s]


In [53]:
order_tr = []

for i in tqdm(range(0,len(df_train_v1))):
    
    mds = df_train_v1['mds_fam_id'].iloc[i]
    dc =  df_train_v1['dc_nbr'].iloc[i]
    dt =  df_train_v1['date'].iloc[i]
    
    order_tr.append(dict_map_tr[(mds,dc,dt)])
    

100%|██████████| 56517/56517 [00:01<00:00, 28320.61it/s]


In [54]:
order_tr = [0.1 if x < 0 else x for x in order_tr]

In [55]:
df_train_v1['pred_order'] = order_tr

/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
#x_train['pred_order'] = x_train['pred_order'] * x_train['vnpk_qty']

In [58]:
df_train_v1['res'] = df_train_v1['order'] - df_train_v1['pred_order']

/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### Time series approach

In [59]:
id_unique = list(df_train_v1['mds_dc'].unique())

In [60]:
dict_tr_te = {}

for i in tqdm(id_unique):
    seq = list(df_train_v1[df_train_v1['mds_dc'] == i]['res'])
    
    if len(seq)!=1:
        model = ExponentialSmoothing(seq).fit()
    else :
        model = ExponentialSmoothing(seq*3).fit()
    
    k = df_test_v1[df_test_v1['mds_dc'] == i].shape[0]
    dict_tr_te[i] = list(model.forecast(k))


  9%|▉         | 40/442 [00:00<00:44,  9.04it/s]/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/statsmodels/tsa/holtwinters.py:712: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
 69%|██████▉   | 307/442 [00:01<00:00, 164.63it/s]/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/statsmodels/tsa/holtwinters.py:712: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/statsmodels/tsa/holtwinters.py:712: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
100%|██████████| 442/442 [00:02<00:00, 176.90it/s]


In [61]:
df_test_v3 = df_test_v1


In [67]:
df_test_v3 = pd.DataFrame()

for i in tqdm(id_unique):
    
    df_temp = df_test_v1[df_test_v1['mds_dc'] == i]
    df_temp['ts_ord'] = dict_tr_te[i]
    
    df_test_v3 =  pd.concat([df_test_v3,df_temp],axis=0)
    
   

  0%|          | 0/442 [00:00<?, ?it/s]/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 442/442 [00:02<00:00, 152.23it/s]


In [78]:
df_test_v3

,mds_fam_id,dc_nbr,date,dept_nbr,ind,week,year,month,mds_dc,dept_dc,...,weekcount1,yearcount1,monthcount1,mds_dccount1,dept_dccount1,dept_mdscount1,mds_weekcount1,order,ts_ord,final_ord
140,105899148.0,6006.0,2019-08-10,46.0,te,32.0,2019.0,8.0,0.0,3.0,...,32.0,32.0,32.0,32.0,18.0,32.0,32.0,31.969309,-0.588532,31.380776
141,105899148.0,6006.0,2019-08-24,46.0,te,34.0,2019.0,8.0,0.0,3.0,...,32.0,32.0,32.0,32.0,18.0,32.0,32.0,31.177563,-0.588532,30.589031
142,105899148.0,6006.0,2019-09-07,46.0,te,36.0,2019.0,9.0,0.0,3.0,...,32.0,32.0,32.0,32.0,18.0,32.0,32.0,30.718068,-0.588532,30.129536
143,105899148.0,6006.0,2019-09-21,46.0,te,38.0,2019.0,9.0,0.0,3.0,...,32.0,32.0,32.0,32.0,18.0,32.0,32.0,31.416503,-0.588532,30.827971
144,105899148.0,6006.0,2019-10-05,46.0,te,40.0,2019.0,10.0,0.0,3.0,...,24.0,32.0,24.0,32.0,18.0,32.0,32.0,30.349884,-0.588532,29.761352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60415,94169512.0,7045.0,2019-09-14,46.0,te,37.0,2019.0,9.0,441.0,302.0,...,32.0,32.0,32.0,4.0,18.0,5.0,7.0,7.991347,-0.652153,7.339194
60416,94169512.0,7045.0,2019-09-21,46.0,te,38.0,2019.0,9.0,441.0,302.0,...,32.0,32.0,32.0,4.0,18.0,5.0,5.0,5.312230,-0.652153,4.660076
60417,94169512.0,7045.0,2019-09-28,46.0,te,39.0,2019.0,9.0,441.0,302.0,...,32.0,32.0,32.0,4.0,18.0,5.0,7.5,7.799787,-0.652153,7.147633
60418,94169512.0,7045.0,2019-10-05,46.0,te,40.0,2019.0,10.0,441.0,302.0,...,24.0,32.0,24.0,4.0,18.0,5.0,16.0,17.681053,-0.652153,17.028900


In [72]:
df_test_v3['final_ord'] = df_test_v3['order'] + df_test_v3['ts_ord']

In [76]:
df_test_v3['final_ord'] = df_test_v3['final_ord'].apply(lambda x: 0.5 if x<0 else x)

In [79]:
dict_map = {}

for i in tqdm(range(0,len(df_test_v3))):
    
    mds = df_test_v3['mds_fam_id'].iloc[i]
    dc =  df_test_v3['dc_nbr'].iloc[i]
    dt =  df_test_v3['date'].iloc[i]
    
    dict_map[(mds,dc,dt)] = df_test_v3['final_ord'].iloc[i]

100%|██████████| 3997/3997 [00:00<00:00, 16974.76it/s]


In [81]:
dict_dt = {}

for i in tqdm(range(0,len(x_test))):
    
    dict_dt[x_test['wm_year_wk_nbr'].iloc[i]] = x_test['date'].iloc[i]

100%|██████████| 3903/3903 [00:00<00:00, 29142.46it/s]


In [82]:
order = []

for i in tqdm(range(0,len(final_test))):
    
    mds = final_test['mds_fam_id'].iloc[i]
    dc =  final_test['dc_nbr'].iloc[i]
    dt =  dict_dt[final_test['wm_year_wk_nbr'].iloc[i]]
    
    order.append(dict_map[(mds,dc,dt)])
    

100%|██████████| 3903/3903 [00:00<00:00, 19995.20it/s]


In [83]:
final_test['predicted_order_qty'] = order
final_test['predicted_order_qty'] = final_test['predicted_order_qty'] * final_test['vnpk_qty']

In [84]:
final_test.to_csv('/Users/s0c02nj/Desktop/merch_pred_win.csv',index = False)